# Setup 

In [1]:
import os 
import pandas as pd 
import numpy as np 

import shap 
import xgboost as xgb 
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder 

# get the folder path from an environment variable 
folder_path = os.environ.get("NFL_DATA_PATH") 

# turn off the pandas warning 
pd.options.mode.chained_assignment = None  # default='warn' 

C:\Users\rjfis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Import Data 

In [2]:
# load the supplementary data for each play 
df_supp = pd.read_csv(f"{folder_path}/supplementary_data.csv") 
df_supp["game_play_key"] = df_supp["game_id"].astype(str) + "-" + df_supp["play_id"].astype(str) 

# load the defender metrics 
df_def = pd.read_csv(f"{folder_path}/defender_metrics.csv") 

# load the combine data 
df_combine = pd.read_csv(f"{folder_path}/combine_data.csv") 

# join the dataframes together 
df_plays = (
    df_def[["game_play_key", "nfl_id", "top_speed_mph", "peak_accel", "separation"]]
    .merge(df_supp[[
        "game_play_key", "week", "pass_result", "route_of_targeted_receiver", 
        "down", "yards_to_go", "team_coverage_type"
    ]], on = "game_play_key", how = "left") 
    .merge(df_combine[["nfl_id", "40yd"]], on = "nfl_id", how = "inner") 
) 

# add completion classification 
df_plays["is_completion"] = np.where(df_plays["pass_result"] == "C", 1, 0) 

# showcase the data 
df_plays.head() 

C:\Users\rjfis\AppData\Local\Temp\ipykernel_25580\2557291801.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_supp = pd.read_csv(f"{folder_path}/supplementary_data.csv")


,game_play_key,nfl_id,top_speed_mph,peak_accel,separation,week,pass_result,route_of_targeted_receiver,down,yards_to_go,team_coverage_type,40yd,is_completion
0,2023090700-101,46137,17.011739,5.667892,1.839674,1,I,CORNER,3,3,COVER_2_ZONE,4.40,0
1,2023090700-1069,53487,7.775418,6.149146,5.281534,1,C,IN,1,10,COVER_3_ZONE,4.60,1
2,2023090700-1154,54486,14.186079,8.192108,3.631116,1,C,CROSS,2,7,COVER_3_ZONE,4.44,1
3,2023090700-1201,54486,5.507555,6.151817,4.118847,1,C,SLANT,2,11,COVER_2_ZONE,4.44,1
4,2023090700-1494,54486,14.244943,3.629354,1.090000,1,I,IN,2,5,COVER_3_ZONE,4.44,0


# Completion Model 

## Data Prep 

In [3]:
# split the data into train and test 
df_train = df_plays.loc[df_plays["week"] <= 13] 
df_test = df_plays.loc[df_plays["week"] > 13] 

# showcase the number of plays in each 
print(f"Number of training plays: {len(df_train.index):,}") 
print(f"Number of testing plays: {len(df_test.index):,}") 

# encode the route_of_targeted_receiver variable 
le1 = LabelEncoder() 
df_train['route_encoded'] = le1.fit_transform(df_train['route_of_targeted_receiver']) 
df_test['route_encoded'] = le1.transform(df_test['route_of_targeted_receiver']) 

# encode the team_coverage_type variable 
le2 = LabelEncoder() 
df_train["coverage_encoded"] = le2.fit_transform(df_train["team_coverage_type"]) 
df_test["coverage_encoded"] = le2.transform(df_test["team_coverage_type"]) 

# define the variables 
predictor_vars = ['40yd', 'route_encoded', 'coverage_encoded'] 
response_var = 'is_completion' 

# subset the train/test features and target variables 
X_train = df_train[predictor_vars]
y_train = df_train[response_var] 
X_test = df_test[predictor_vars]
y_test = df_test[response_var] 

Number of training plays: 5,400
Number of testing plays: 1,711


## Model Training 

In [4]:
# Initialize and train the XGBoost model 
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss'
)

# fit the model 
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred_train = xgb_model.predict(X_train) 
y_pred_test = xgb_model.predict(X_test) 

# Evaluate model
print(f"Training Accuracy: {accuracy_score(y_train, y_pred_train):.1%}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_test):.1%}") 

Training Accuracy: 70.2%
Test Accuracy: 66.5%


## SHAP Values 

In [15]:
# Calculate SHAP values for test data
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test) 

# put the shap values into a dataframe 
df_shap = pd.DataFrame({
    "game_play_key": df_test["game_play_key"].values, 
    "nfl_id": df_test["nfl_id"].values, 
    "40yd": df_test["40yd"].values, 
    "route_of_targeted_receiver": df_test["route_of_targeted_receiver"].values, 
    "def_coverage_type": df_test["team_coverage_type"].values, 
    "shap_40yd": shap_values[:,0], 
    "shap_route_encoded": shap_values[:,1] 
}) 

# add a distinction for "high-impact" plays 
df_shap["high_impact"] = np.where(df_shap["shap_40yd"] < -0.1, 1, 0) 

# summarize the SHAP values for 40 yard time 
shap_40yd = df_shap["shap_40yd"].to_list() 
pcts = [5, 10, 25, 50, 75, 90, 95]
vals = np.percentile(shap_40yd, pcts) 
print("Percentiles for 40 yd SHAP values: ")
for p, v in zip(pcts, vals): 
    print(f"  {p}th: {v:.4f}") 

# showcase the results 
df_shap.sort_values("shap_40yd", ascending = True).head(15) 

Percentiles for 40 yd SHAP values: 
  5th: -0.2025
  10th: -0.1565
  25th: -0.0753
  50th: 0.0068
  75th: 0.0755
  90th: 0.1254
  95th: 0.2214


,game_play_key,nfl_id,40yd,route_of_targeted_receiver,def_coverage_type,shap_40yd,shap_route_encoded,high_impact
18,2023120700-2959,37078,4.31,SCREEN,COVER_3_ZONE,-0.515352,0.329524,1
549,2023121700-878,53458,4.31,GO,COVER_3_ZONE,-0.513811,-1.090534,1
974,2023122403-1734,46124,4.32,HITCH,COVER_3_ZONE,-0.490700,0.331150,1
984,2023122403-2414,53458,4.31,HITCH,COVER_3_ZONE,-0.490700,0.331150,1
1487,2023123106-983,46124,4.32,HITCH,COVER_3_ZONE,-0.490700,0.331150,1
1291,2023122800-3391,46073,4.32,HITCH,COVER_3_ZONE,-0.490700,0.331150,1
1464,2023123106-134,46124,4.32,OUT,COVER_3_ZONE,-0.464618,0.052837,1
741,2023121709-2997,46124,4.32,OUT,COVER_3_ZONE,-0.464618,0.052837,1
529,2023121700-2443,54622,4.33,GO,COVER_3_ZONE,-0.433237,-1.156756,1
777,2023121710-4748,46073,4.32,OUT,COVER_2_ZONE,-0.397069,0.049284,1


# Example Setups 

## 40 Time Stats 

In [6]:
# calculate the average 40 time 
avg_40yd = df_shap["40yd"].mean() 
print(f"Average 40 Yard Dash Time: {avg_40yd:.2f} seconds") 

Average 40 Yard Dash Time: 4.51 seconds


## calc_completion_prob 

In [7]:
def calc_completion_prob(game_play_key, avg_40 = None): 

    # filter to the given play 
    play_features = X_test.loc[df_test["game_play_key"] == game_play_key].to_dict(orient='records')
    
    # check if play_features is empty 
    if len(play_features) == 0:
        print(f"No data found for game_play_key: {game_play_key}") 
        comp_prob = None 
    
    # otherwise, continue 
    else:
        play_features = play_features[0] 

        # replace the 40 time with the average if provided 
        if avg_40 is not None: 
            play_features['40yd'] = avg_40 
        
        # calculate the completion probability 
        comp_prob = xgb_model.predict_proba(
            pd.DataFrame(
                [[play_features['40yd'], play_features['route_encoded'], play_features['coverage_encoded']]], 
                columns = ["40yd", "route_encoded", "coverage_encoded"]
            )
        )[:, 1][0] 
    
    return comp_prob 

# test the function 
play_key = "2023121700-878"
print(calc_completion_prob(play_key)) 
print(calc_completion_prob(play_key, avg_40 = avg_40yd)) 

0.29401815
0.4279461


## compare_scenarios 

In [17]:
def compare_scenarios(game_play_key): 

    # calculate the probabilities 
    prob_actual = calc_completion_prob(game_play_key) 
    prob_avg = calc_completion_prob(game_play_key, avg_40 = avg_40yd) 

    # get the defender attributes for the play 
    defender_id = df_test.loc[df_test["game_play_key"] == game_play_key, "nfl_id"].values[0] 
    defender_40 = df_test.loc[df_test["game_play_key"] == game_play_key, "40yd"].values[0] 

    # showcase the results 
    print(f"\nFor play {game_play_key}:") 
    print(f"  Completion probability against {defender_id}: {prob_actual:.1%} (40 time = {defender_40:.2f} seconds)")
    print(f"  Completion probability against average defender: {prob_avg:.1%} (avg 40 time = {avg_40yd:.2f} seconds)") 

# test the function 
play_key = "2023121700-878" 
compare_scenarios(play_key)


For play 2023121700-878:
  Completion probability against 53458: 29.4% (40 time = 4.31 seconds)
  Completion probability against average defender: 42.8% (avg 40 time = 4.51 seconds)


# Metric Comparison 

In [9]:
# aggregate by the "high-impact" distinction 
df_metrics = (
    df_shap[["game_play_key", "high_impact"]]
    .merge(df_plays, on = ["game_play_key"], how = "inner") 
) 

# aggregate to compare 
df_sums = df_metrics.groupby("high_impact").agg(
    num_plays = ("game_play_key", "count"), 
    avg_40yd = ("40yd", "mean"), 
    avg_top_speed = ("top_speed_mph", "mean"), 
    avg_peak_accel = ("peak_accel", "mean"), 
    avg_separation = ("separation", "mean"), 
    completion_rate = ("is_completion", "mean") 
).reset_index()  

# put in a dictionary 
metrics = {
    "top": df_sums.loc[df_sums["high_impact"] == 1].to_dict(orient = "records")[0], 
    "not": df_sums.loc[df_sums["high_impact"] == 0].to_dict(orient = "records")[0] 
} 

# showcase the number of plays 
print(f"Number of high-impact plays: {metrics['top']['num_plays']:,} out of {len(df_plays.index):,} total")

# calculate the percent differences 
pct_top_speed = (metrics["top"]["avg_top_speed"] / metrics["not"]["avg_top_speed"]) - 1 
pct_peak_accel = (metrics["top"]["avg_peak_accel"] / metrics["not"]["avg_peak_accel"]) - 1 
pct_separation = 1 - (metrics["top"]["avg_separation"] / metrics["not"]["avg_separation"]) 
pct_completion = 1 - (metrics["top"]["completion_rate"] / metrics["not"]["completion_rate"]) 

# showcase the metrics 
print("\nOn high-impact plays, we saw the defenders...") 
print(f"- Have a {pct_top_speed:.2%} higher average top speed ({metrics['top']['avg_top_speed']:.2f} mph vs {metrics['not']['avg_top_speed']:.2f} mph)") 
print(f"- Have a {pct_peak_accel:.2%} higher average peak acceleration ({metrics['top']['avg_peak_accel']:.2f} vs {metrics['not']['avg_peak_accel']:.2f})") 
print(f"- Allow {pct_separation:.2%} less average separation ({metrics['top']['avg_separation']:.2f} yards vs {metrics['not']['avg_separation']:.2f} yards)") 
print(f"- Allow a {pct_completion:.2%} lower completion rate ({metrics['top']['completion_rate']:.2%} vs {metrics['not']['completion_rate']:.2%})") 

Number of high-impact plays: 334 out of 7,111 total

On high-impact plays, we saw the defenders...
- Have a 5.27% higher average top speed (13.51 mph vs 12.83 mph)
- Have a 0.56% higher average peak acceleration (5.00 vs 4.97)
- Allow 15.84% less average separation (2.70 yards vs 3.21 yards)
- Allow a 8.98% lower completion rate (61.08% vs 67.10%)


## Breakout by Route 

In [10]:
# aggregate by the route of the targeted receiver 
df_sums = df_metrics.groupby("route_of_targeted_receiver").agg(
    num_plays = ("game_play_key", "count"),
    high_impact = ("high_impact", "sum"), 
    high_impact_pct = ("high_impact", "mean"), 
    avg_top_speed = ("top_speed_mph", "mean"), 
    avg_peak_accel = ("peak_accel", "mean"), 
    avg_separation = ("separation", "mean"), 
    completion_rate = ("is_completion", "mean") 
).reset_index() 

# showcase the results 
df_sums.sort_values("high_impact_pct", ascending = False) 

,route_of_targeted_receiver,num_plays,high_impact,high_impact_pct,avg_top_speed,avg_peak_accel,avg_separation,completion_rate
4,GO,171,70,0.409357,16.607058,4.108824,1.872627,0.421053
5,HITCH,340,88,0.258824,10.332155,6.150686,2.818367,0.726471
0,ANGLE,51,13,0.254902,10.457965,5.458929,4.031350,0.647059
8,POST,121,25,0.206612,14.174414,4.284668,2.594816,0.528926
10,SLANT,125,24,0.192000,11.340290,5.000249,2.231437,0.632000
9,SCREEN,32,6,0.187500,10.508263,5.826258,6.357003,0.875000
11,WHEEL,11,2,0.181818,15.182443,4.650947,3.163487,0.545455
6,IN,141,20,0.141844,12.376235,5.400217,2.653728,0.702128
1,CORNER,65,9,0.138462,15.779334,4.685017,2.655702,0.553846
7,OUT,273,35,0.128205,13.491337,5.344445,2.856775,0.677656


## Breakout by Coverage 

In [12]:
# aggregate by the route of the targeted receiver 
df_sums = df_metrics.groupby("team_coverage_type").agg(
    num_plays = ("game_play_key", "count"),
    high_impact = ("high_impact", "sum"), 
    high_impact_pct = ("high_impact", "mean"), 
    avg_top_speed = ("top_speed_mph", "mean"), 
    avg_peak_accel = ("peak_accel", "mean"), 
    avg_separation = ("separation", "mean"), 
    completion_rate = ("is_completion", "mean")  
).reset_index() 

# showcase the results 
df_sums.sort_values("high_impact_pct", ascending = False) 

,team_coverage_type,num_plays,high_impact,high_impact_pct,avg_top_speed,avg_peak_accel,avg_separation,completion_rate
4,COVER_3_ZONE,541,127,0.234750,12.570767,5.223612,3.395772,0.683919
2,COVER_2_MAN,40,9,0.225000,13.750305,4.282669,2.462226,0.525000
1,COVER_1_MAN,394,86,0.218274,14.398287,4.472503,2.305337,0.591371
0,COVER_0_MAN,75,16,0.213333,13.649793,4.661442,3.015023,0.626667
5,COVER_4_ZONE,281,50,0.177936,12.825436,4.918971,3.179812,0.658363
7,PREVENT,8,1,0.125000,13.232359,4.968738,3.250605,0.750000
3,COVER_2_ZONE,193,24,0.124352,11.433521,5.370151,4.000623,0.740933
6,COVER_6_ZONE,179,21,0.117318,12.373750,5.271044,3.089605,0.687151
